# Question: Does the company promote diversity and inclusion?

Although many ESG issues may not be financially material to all industries, we believe that diversity and inclusion of all people -- regardless of gender, race, sexual orientation, religion, background, and other similar traits -- is one that can be viewed as material across the board.

Embracing different types of people is the moral thing to do, but it's also the best financial thing to do. In 2018, management consulting firm McKinsey & Co. released the report "Why Diversity Matters"; it examined data sets from 366 public companies in the U.S., Canada, Latin America, and the U.K. Companies in the top quartile for gender, racial, or ethnic diversity were more likely to generate financial returns above the national medians for their industry; the converse was also true. Meanwhile, McKinsey concluded that "diversity is probably a competitive differentiator that shifts market share toward more diverse companies over time."

Why? Plenty of data shows that diverse groups make better decisions than homogeneous ones. Narrow-minded in-group thinking, in which members are more likely to have similar experiences and viewpoints and come to the same conclusions, can be terrible for decision-making. Meanwhile, groups in which many different life experiences and perspectives are represented help set the stage for better decisions.

Underlining this principle is the correlation between what a company does and who runs it. For example, if a retailer sells jewelry aimed at female consumers, having an entirely male management team and board of directors would seem absurdly mismatched -- it would result in missing valuable perspectives about the very people it's aiming to attract as customers.

In ESG investing, we seek positive elements like a transparent workforce-diversity disclosure (some companies don't disclose their staff makeup at all); composition of its board of directors and executive leadership; and internal programs and policies that foster or support diversity -- adequate paid family leave, professional programs for people of color, and fair hiring practices.

Companies with poor diversity statistics, or lawsuits related to discrimination and harassment, will be noted for their heightened risk and probably disqualified from inclusion.

# Data Sources

- SEC Reports

# Imports

In [57]:
import sec_edgar_downloader
from sec_edgar_downloader import Downloader
import re
import os

# Getting Data

In [3]:
d = Downloader()
d.get('DEF 14A', 
      'AAPL', 
      after='2017-01-01',
      download_details=False,
      query = 'diversity inclusion')

2

In [55]:
text_files = []
for root, dirs, files in os.walk("/Users/MichaelWirtz/Desktop/portfolio/programming_projects/long_term_trading_bot/archive/sec-edgar-filings/AAPL"):
    for file in files:
        if file.endswith(".txt"):
             text_files.append(os.path.join(root, file))

In [56]:
text_files

['/Users/MichaelWirtz/Desktop/portfolio/programming_projects/long_term_trading_bot/archive/sec-edgar-filings/AAPL/DEF 14A/0001193125-21-001987/full-submission.txt',
 '/Users/MichaelWirtz/Desktop/portfolio/programming_projects/long_term_trading_bot/archive/sec-edgar-filings/AAPL/DEF 14A/0001193125-19-004664/full-submission.txt']

In [6]:
first = open('./sec-edgar-filings/AAPL/DEF 14A/0001193125-19-004664/full-submission.txt').read()


In [21]:
valids = re.sub(r"[^a-z]+", ' ', first)

In [22]:
valids.count('diversity') + valids.count('inclusion')

9